### Import all relevant dependencies

In [2]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from pathlib import Path
import yaml
from src.minimizer.minimizer_library.differential_evolution_save import DifferentialEvolutionParallel
from src.visualization.plotting_functions import *
from src.visualization.plotting_convergence import *
from src.statistical_models.statistical_model_library.fcs_gaussian_noise_model import FCSGaussianNoiseModel
from src.utils.experiment_serialization import import_all_experiments, import_experiment_results
from src.math_utils.experiment_metrics import evaluate_full_metrics
from src.model.hahn_stack_model import HahnStackModel
from src.math_utils.derivatives.numeric_derivative_calculator import NumericDerivativeCalculator
from src.model.parameter_set.hahn_parameter_set import HahnParameterSet
from src.math_utils.scaler.hahn_parameter_scaler import HahnParameterScaler

### Define the Experiment Metadata
- bounds for the free parameters
- operating conditions
- experiment repetitions
- true parameter set
- Experiment variance
- current values

Naming:
unscaled/ rescaled: actual physical values
scaled: scaled value between 0 & 1

In [2]:
path = "../data/config/reduced_example_config_workflow.yaml"

with open(path, "r") as f:
    config = yaml.safe_load(f)

number_designs = config["number_designs"] #Amount of LH Designs
n_rep = config["n_rep"] #Amount of experiment repetitions
n_current_values = config["n_current_values"] #Amount of individual current values
sigma = config["sigma"] #experiment variance (10mV variance in repeated experiments)

# lower and upper bounds for operating conditions
upper_bounds_operating_conditions = np.array(config["upper_bounds_operating_conditions"])
lower_bounds_operating_conditions = np.array(config["lower_bounds_operating_conditions"])

I_S_array = np.linspace(1, 600, n_current_values) # initialize applicable current array request

# select parameter values and names to be analyzed
unscaled_theta_true = np.array([list(HahnParameterSet().free_parameters.values())[i] for i in [1, 2, 4]])
names_theta = [list(HahnParameterSet().free_parameters.keys())[i] for i in [1, 2, 4]]

# initialize lower and upper bounds for free parameter values
unscaled_upper_bounds_free_params = np.array(config["unscaled_upper_bounds_free_params"], dtype=float)
unscaled_lower_bounds_free_params = np.array(config["unscaled_lower_bounds_free_params"], dtype=float)

print(names_theta, unscaled_lower_bounds_free_params, unscaled_theta_true, unscaled_upper_bounds_free_params)

['j_0_ref', 'r_el', 'D_GDL_ref'] [1.e+01 1.e-06 1.e-06] [2.1308e+03 4.2738e-06 8.6266e-06] [5.e+03 1.e-03 1.e-05]


### Define Scaler and scale parameter values as well as  bounds of operating conditions and parameters
Scalers are saved in variable "scaler" and handed over to stack model.

In [3]:
scaler = HahnParameterScaler() # define scaler

# Stack bounds of free parameters
free_parameter_bounds = np.vstack([
    unscaled_lower_bounds_free_params,
    unscaled_upper_bounds_free_params
]).T

# Stack operating condition bounds (rows = condition, columns = [min, max])
operating_condition_bounds = np.vstack([
    lower_bounds_operating_conditions,
    upper_bounds_operating_conditions
]).T

# Determine current range for scaling
current_bounds = np.array([[I_S_array.min(), I_S_array.max()]])

scaled_theta_true = scaler.scale_theta(unscaled_theta_true, free_parameter_bounds)

# scale bounds of operating conditions to hand over for Experimental designs incl. LHC
scaled_upper_bounds = scaler.scale_params(upper_bounds_operating_conditions, operating_condition_bounds)
scaled_lower_bounds = scaler.scale_params(lower_bounds_operating_conditions, operating_condition_bounds)

# scale bounds of free parameters for initializing Model
scaled_lower_bounds_theta, _ = scaler.scale(unscaled_lower_bounds_free_params, free_parameter_bounds)
scaled_upper_bounds_theta, _ = scaler.scale(unscaled_upper_bounds_free_params, free_parameter_bounds)

crlb_factor = unscaled_upper_bounds_free_params-unscaled_lower_bounds_free_params # offset for reformating the CRLB
unit_factors = np.array([1e-4, 1e4, 1e4]) #[1e-3, 1e-4, 1e4, 1e4, 1e4, 1]) factors to reformat the units from SI to applicable industry standard units

# Print scaling results of scaled true parameters
print("Scaled theta:", scaled_theta_true)
print("Rescaled theta:", unscaled_theta_true)

Scaled theta: [0.42501002 0.00327708 0.8474    ]
Rescaled theta: [2.1308e+03 4.2738e-06 8.6266e-06]


### Define the parametric function including handover of bounds

In [4]:
hahn_fc_model = HahnStackModel(parameter_set=HahnParameterSet(free_parameters=names_theta)) #Instantiate the generic Hahn Model
calculator = NumericDerivativeCalculator(hahn_fc_model, scaler) #Instantiate the derivation calculator function

#Initialize statistical model function
statistical_model = FCSGaussianNoiseModel(model_function=hahn_fc_model,
                                          der_function=calculator,
                                          lower_bounds_x=scaled_lower_bounds,
                                          upper_bounds_x=scaled_upper_bounds,
                                          lower_bounds_theta=scaled_lower_bounds_theta,
                                          upper_bounds_theta=scaled_upper_bounds_theta,
                                          sigma=sigma,
                                          scaler = scaler,)

#Initialize blackbox function returning noised experiment results
def blackbox_model(x):
    return statistical_model.random(theta=scaled_theta_true, x=x)

### Import Base LHC Designs

In [5]:
import_path = ".." / Path("data") / "experimental_designs" / "lhc"
df_exp_input = import_experiment_results(import_path/"lhc.csv")

# prepare LHC data for further analysis
x_designs = df_exp_input[['Pressure', 'Temperature', 'Stoichiometry', 'Current']].to_numpy()
y_designs = df_exp_input[['Voltage']].to_numpy()

# Slice Designs (n=25000) of the repetitions (n=250) for optimal experiment calculation
x0_LH_design = x_designs[:number_designs * n_current_values:]

## define minimizer for theta estimation on imported experiments

In [6]:
iterations = 1000

# Without noise, the DE optimizer converges towards theta_true
minimizer = DifferentialEvolutionParallel(maxiter=iterations, save_intermediate=True)

# Import additional Experiments for optimality criteria

In [7]:
# Load all_additional experiments (LHC + others)
df_all = import_all_experiments(".." / Path("data") / "experimental_designs" / "other")
experiments = {name: group for name, group in df_all.groupby("SourceFile")}

# Designs singular with 5 additional Experiments
in total n_number_new_designs (default: 5) * n_current_values * n_rep = 5000

In [8]:
x_designs_all = {
    "LH_new": experiments["LH_new"][['Pressure','Temperature','Stoichiometry','Current']].to_numpy(),
    "a_design": experiments["a_design"][['Pressure','Temperature','Stoichiometry','Current']].to_numpy(),
    "d_design": experiments["d_design"][['Pressure','Temperature','Stoichiometry','Current']].to_numpy(),
    "pi_design": experiments["pi_design"][['Pressure','Temperature','Stoichiometry','Current']].to_numpy(),
}

y_designs_all = {
    "LH_new": experiments["LH_new"][['Voltage']].to_numpy(),
    "a_design": experiments["a_design"][['Voltage']].to_numpy(),
    "d_design": experiments["d_design"][['Voltage']].to_numpy(),
    "pi_design": experiments["pi_design"][['Voltage']].to_numpy(),
}

#### Stack Designs to form 25LHC+5OED Arrays

In [9]:
from src.utils.experiment_serialization import combine_experiments

cols_x = ['Pressure','Temperature','Stoichiometry','Current']
cols_y = ['Voltage']
design_names = ["LH_new", "a_design", "d_design", "pi_design"]

x_designs_all_stacked = {
    name: np.vstack([x_designs, experiments[name][cols_x].to_numpy()])
    for name in design_names
}

y_designs_all_stacked = {
    name: np.vstack([y_designs, experiments[name][cols_y].to_numpy().reshape(-1, 1)])
    for name in design_names
}

x_designs_all_stacked_new = {
    name: combine_experiments(x_designs, experiments[name][cols_x].to_numpy(), n_rep=n_rep)
    for name in design_names
}

y_designs_all_stacked_new = {
    name: combine_experiments(y_designs, experiments[name][cols_y].to_numpy().reshape(-1, 1), n_rep=n_rep)
    for name in design_names
}

#test print if the length of the experiment arrays is as expected. should be: (number_designs + n_new_designs) * n_current_values * n_rep
for k in x_designs_all_stacked:
    print(k, x_designs_all_stacked[k].shape, y_designs_all_stacked[k].shape)

for k in x_designs_all_stacked_new:
    print(k, x_designs_all_stacked_new[k].shape, y_designs_all_stacked_new[k].shape)

LH_new (3100, 4) (3100, 1)
a_design (3100, 4) (3100, 1)
d_design (3100, 4) (3100, 1)
pi_design (3100, 4) (3100, 1)
LH_new (3100, 4) (3100, 1)
a_design (3100, 4) (3100, 1)
d_design (3100, 4) (3100, 1)
pi_design (3100, 4) (3100, 1)


# All Designs stacked with 25LH + 5 additional Experiments
in total (number_designs (default: 25) + n_number_new_designs (default: 5)) * n_current_values * n_rep = 30000

In [ ]:
results_df = evaluate_full_metrics(
    statistical_model=statistical_model,
    scaled_theta_true=scaled_theta_true,
    x_designs=x_designs_all_stacked_new,
    y_designs=y_designs_all_stacked_new,
    n_rep=n_rep,
    minimizer=minimizer,
    scaler=scaler,
)

Estimating thetas:   3%|▎         | 3/100 [04:36<2:32:29, 94.32s/it]

In [11]:
crlb_factor = unscaled_upper_bounds_free_params-unscaled_lower_bounds_free_params # offset for reformating the CRLB
unit_factors = np.array([1e-4, 1e4, 1e4]) # factors to reformat the units from SI to applicable industry standard units

# Rescale parameter-dependent metrics using HahnParameterScaler
for idx, row in results_df.iterrows():
    est_theta_rescaled = scaler.rescale_theta(row["est_theta"])
    diag_CRLB_rescaled = row["diag_CRLB"]*crlb_factor*crlb_factor*unit_factors*unit_factors
    crlb_rescaled = row["CRLB"]*crlb_factor*crlb_factor*unit_factors*unit_factors  # matrix diag only

    scales = np.array([s.scale_[0] for s in scaler.theta_scalers])
    var_theta_rescaled = row["var_theta"] * (scales**2)
    std_rescaled = np.sqrt(var_theta_rescaled)

    print(f"\n=== Experiment: {row['Experiment']} ===")
    print(f"det(FIM): {row['det_FIM']:.3e}")
    print(f"est_theta (scaled): {row['est_theta']}")
    print(f"est_theta (rescaled): {est_theta_rescaled}")
    print(f"var_theta (scaled): {row['var_theta']}")
    print(f"var_theta (rescaled): {var_theta_rescaled}")
    print(f"diag_CRLB (scaled): {row['diag_CRLB']}")
    print(f"diag_CRLB (rescaled): {diag_CRLB_rescaled}")
    print(f"standard deviation (rescaled): {std_rescaled}")
    print(f"rel_std: {row['rel_std']}")
    print(f"rel_bias: {row['rel_bias']}")
    print(f"rel_rmse: {row['rel_rmse']}")


=== Experiment: LH_new ===
det(FIM): 4.021e+13
est_theta (scaled): [0.42527443 0.00328335 0.8578292 ]
est_theta (rescaled): [2.13211940e+03 4.28006808e-06 8.72046280e-06]
var_theta (scaled): [1.49781368e-03 4.46968917e-08 3.57968138e-03]
var_theta (rescaled): [6.01529184e-11 4.47864198e-02 4.41935973e+07]
diag_CRLB (scaled): [1.29991041e-03 3.62717306e-08 3.60854225e-03]
diag_CRLB (rescaled): [3.23678991e-04 3.61992234e-06 2.92291922e-05]
standard deviation (rescaled): [7.75583125e-06 2.11628022e-01 6.64782651e+03]
rel_std: [0.08477883 0.05800518 0.07002693]
rel_bias: [0.00062212 0.00191462 0.01230729]
rel_rmse: [0.09060613 0.06421888 0.07132072]

=== Experiment: a_design ===
det(FIM): 1.214e+16
est_theta (scaled): [0.42530186 0.00327814 0.84747514]
est_theta (rescaled): [2.13225627e+03 4.27486109e-06 8.62727629e-06]
var_theta (scaled): [1.56057869e-03 2.24518219e-08 9.51826498e-06]
var_theta (rescaled): [6.26735914e-11 2.24967930e-02 1.17509444e+05]
diag_CRLB (scaled): [1.22802705e-0

### Save Experiment results and theta estimations

In [12]:
from pathlib import Path
# save estimated thetas
root = Path.cwd().parent   # this will be oed_fuel_cell_model/
data_path = root / "data" / "experimental_designs" / "other"

# Save DataFrame
results_df.to_pickle(data_path / "combined_experiments_results_df.pkl")

In [3]:
plot_multiple_runs("./minimizer_logs/convergence_20251011_061259.csv")

FileNotFoundError: [Errno 2] No such file or directory: './minimizer_logs/convergence_20251011_061259.csv'